In [ ]:
from pynq import Overlay, allocate, PL, MMIO
import cv2
import numpy as np
import time

BITSTREAM_PATH = "/home/xilinx/pynq/overlays/binarizer_v2/binarizer_v2.bit"
WIDTH, HEIGHT = 1280, 720
CHANNELS = 4

In [ ]:
PL.reset()
overlay = Overlay(BITSTREAM_PATH)

In [ ]:
def display_setup(ol):
    # 1. Access VDMA via MMIO
    vdma_phys = ol.ip_dict['axi_vdma_0']['phys_addr']
    vdma_mmio = MMIO(vdma_phys, 65536)
    
    MM2S_DMACR      = 0x00  # Control Register
    MM2S_START_ADDR = 0x5C  # Start Address 1
    MM2S_STRIDE     = 0x58  # Stride (Bytes per line)
    MM2S_HSIZE      = 0x54  # Horizontal Size (Bytes per line)
    MM2S_VSIZE      = 0x50  # Vertical Size (Rows) - Writing this STARTS the frame

    # 2. Setup Frame Buffer
    frame_buffer = allocate(shape=(HEIGHT, WIDTH, CHANNELS), dtype=np.uint8)
    
    # 3. Configure VDMA (Read Channel)
    # https://docs.amd.com/r/en-US/pg020_axi_vdma/General-Use-Cases
    # bit 0 - Run/Stop
    # bit 1 - Circular Park Mode
    # bit 2 - Reset
    # bit 3 - Genlock Enable
    # bit 4 - Genlock Src
    vdma_mmio.write(MM2S_DMACR, 0x4)
    time.sleep(0.1) # Give it a moment to reset

    # 4. CONFIGURE the Register Settings
    # A. Set the Frame Buffer Address (Where to read pixels from)
    # We write the same address to multiple registers to be safe.
    vdma_mmio.write(MM2S_START_ADDR, frame_buffer.device_address)
    vdma_mmio.write(MM2S_START_ADDR + 4, frame_buffer.device_address) 
    vdma_mmio.write(MM2S_START_ADDR + 8, frame_buffer.device_address)

    # B. Set Stride (Total width of one line in memory)
    # 1280 pixels * 4 bytes/pixel = 5120 bytes
    stride = 1280 * 4
    vdma_mmio.write(MM2S_STRIDE, stride)

    # C. Set Horizontal Size (Active width of line)
    vdma_mmio.write(MM2S_HSIZE, stride)

    # D. Set Control Register (Run + Circular Mode)
    # Bit 0 = 1 (Run)
    # Bit 1 = 1 (Circular Park - Loop on this buffer)
    vdma_mmio.write(MM2S_DMACR, 0x3)

    # E. Set Vertical Size (Height) -> THIS TRIGGER START
    # 720 lines
    vdma_mmio.write(MM2S_VSIZE, 720)
    
    # 5. Enable VTC Generation (Control Register 0x00)
    # https://docs.amd.com/r/en-US/pg016_v_tc/Register-Space
    # https://docs.amd.com/r/en-US/pg016_v_tc/Register-Descriptions
    # Bit 0 = 1 (Enable)
    # Bit 1 = 1 (Generation Enable)
    vtc_phys = ol.ip_dict['v_tc_0']['phys_addr']
    vtc_mmio = MMIO(vtc_phys, 0x100)
    vtc_mmio.write(0x00, vtc_mmio.read(0x00) | 0x3)
    
    print("Display (VDMA + VTC) initialized.")
    return frame_buffer

frame_buffer = display_setup(overlay)

In [ ]:
def run_binarizer_ip(dma, image_path):
    img = cv2.imread(image_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h, w = img_gray.shape
    
    # Flatten and pad for 32-bit alignment (4 bytes)
    flat_data = img_gray.flatten().astype(np.uint8)
    pad = (4 - (len(flat_data) % 4)) % 4
    if pad > 0:
        flat_data = np.pad(flat_data, (0, pad), 'constant')

    # Allocate DMA Buffers
    in_buf = allocate(shape=(len(flat_data),), dtype=np.uint8)
    out_buf = allocate(shape=(len(flat_data),), dtype=np.uint8)
    
    # Transfer
    in_buf[:] = flat_data
    dma.recvchannel.transfer(out_buf)
    dma.sendchannel.transfer(in_buf)
    dma.recvchannel.wait()
    dma.sendchannel.wait()
    
    # Format Output
    result = np.array(out_buf[:-pad] if pad > 0 else out_buf)
    out_img = result.reshape((h, w))
    
    # Free DMA memory
    # in_buf.freeback()
    # out_buf.freeback()
    
    return out_img

processed_image = run_binarizer_ip(overlay.axi_dma_0, '/home/xilinx/images/ragnar.png')

In [ ]:
def update_hdmi_display(image, fb):
    # Ensure RGBA for VDMA
    if len(image.shape) == 2:
        image_rgba = cv2.cvtColor(image, cv2.COLOR_GRAY2RGBA)
    else:
        image_rgba = cv2.cvtColor(image, cv2.COLOR_BGR2RGBA)
        
    # Resize to match HDMI settings
    resized = cv2.resize(image_rgba, (WIDTH, HEIGHT), interpolation=cv2.INTER_AREA)
    
    # Write to hardware-accessible memory
    fb[:] = resized
    fb.flush()
    print("Frame buffer updated and flushed to hardware.")

In [ ]:
def update_hdmi_display_centered(image, fb):
    """
    Resizes image to fit 1280x720 without distortion.
    Adds black bars (letterboxing/pillarboxing) to maintain aspect ratio.
    """
    # 1. Ensure RGBA for VDMA
    if len(image.shape) == 2:
        img_rgba = cv2.cvtColor(image, cv2.COLOR_GRAY2RGBA)
    else:
        img_rgba = cv2.cvtColor(image, cv2.COLOR_BGR2RGBA)
        
    h, w = img_rgba.shape[:2]
    
    # 2. Calculate Scale Factor
    # We want to fit the image inside WIDTHxHEIGHT
    scale_w = WIDTH / w
    scale_h = HEIGHT / h
    scale = min(scale_w, scale_h) # Use the smaller scale to ensure it fits
    
    new_w = int(w * scale)
    new_h = int(h * scale)
    
    # 3. Resize the image with the calculated scale
    resized_img = cv2.resize(img_rgba, (new_w, new_h), interpolation=cv2.INTER_AREA)
    
    # 4. Create a black canvas (the "bars")
    canvas = np.zeros((HEIGHT, WIDTH, CHANNELS), dtype=np.uint8)
    canvas[:, :, 3] = 255 # Set Alpha channel to fully opaque
    
    # 5. Calculate offsets to center the image
    x_offset = (WIDTH - new_w) // 2
    y_offset = (HEIGHT - new_h) // 2
    
    # 6. Place resized image onto the center of the canvas
    canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_img
    
    # 7. Update hardware buffer
    fb[:] = canvas
    fb.flush()
    print(f"Display updated: Scaled {w}x{h} to {new_w}x{new_h} with centering.")

In [ ]:
update_hdmi_display_centered(processed_image, frame_buffer)
# update_hdmi_display(processed_image, frame_buffer)

In [ ]:
def cleanup_hardware(fb, ol):
    # 1. Stop VDMA to prevent it from reading invalid memory
    vdma_phys = ol.ip_dict['axi_vdma_0']['phys_addr']
    vdma_mmio = MMIO(vdma_phys, 65536)
    vdma_mmio.write(0x00, 0x4) # Reset/Stop
    
    # 2. Free the hardware-allocated buffer
    fb.freeback()
    
    # 3. Reset the PL (Optional but recommended)
    PL.reset()
    
    print("Hardware resources released and VDMA stopped.")

# To safely exit, uncomment the line below:
# cleanup_hardware(frame_buffer, overlay)